In [ ]:
def log_progress(sequence, every=None, size=None, name='Items'):
    is_iterator = False
    if size is None:
        try:
            size = len(sequence)
        except TypeError:
            is_iterator = True
    if size is not None:
        if every is None:
            if size <= 200:
                every = 1
            else:
                every = int(size / 200)     # every 0.5%
    else:
        assert every is not None, 'sequence is iterator, set every'

    if is_iterator:
        progress = IntProgress(min=0, max=1, value=1)
        progress.bar_style = 'info'
    else:
        progress = IntProgress(min=0, max=size, value=0)
    label = HTML()
    box = VBox(children=[label, progress])
    display(box)

    index = 0
    try:
        for index, record in enumerate(sequence, 1):
            if index == 1 or index % every == 0:
                if is_iterator:
                    label.value = '{name}: {index} / ?'.format(
                        name=name,
                        index=index
                    )
                else:
                    progress.value = index
                    label.value = u'{name}: {index} / {size}'.format(
                        name=name,
                        index=index,
                        size=size
                    )
            yield record
    except:
        progress.bar_style = 'danger'
        raise
    else:
        progress.bar_style = 'success'
        progress.value = index
        label.value = "{name}: {index}".format(
            name=name,
            index=str(index or '?')
        )

In [ ]:
import numpy as np
import noise

from matplotlib import pyplot as plt
from matplotlib import colors as col
from mpl_toolkits.mplot3d import Axes3D

from ipywidgets import IntProgress, HTML, VBox
from IPython.display import display
import ipywidgets as wdg

from plotly import graph_objs as plgo
from plotly import offline as ploff
ploff.init_notebook_mode(connected=False)

In [ ]:
res = 100
base = np.linspace(0, 5, res)
x, y = np.meshgrid(base, base)
# final_func = np.vectorize(lambda x, y: noise.pnoise2(x, y, octaves=4))
# land = final_func(x, y)
land = np.zeros(x.shape)
x_dir = np.zeros((res, res-1))
x_dir2 = np.zeros((res-1, res-1))
y_dir = np.zeros((res-1, res))
y_dir2 = np.zeros((res-1, res-1))

# water = np.zeros(land.shape)
# water[15:86,15:86] = 5 - land[15:86,15:86]
water = 10 * np.e ** -((x-2.5)**2 + (y-2.5)**2)

In [ ]:
def plot(x, y, land, water):
    mask = ~water.astype(bool)
#     mask = water < 0.0001
    layer = land + water
    layer[mask] = None
    ploff.iplot(
        plgo.Figure({
            'data': [
                plgo.Surface({
                    'name': 'land',
                    'x': x,
                    'y': y,
                    'z': land,
                    'colorscale': [(0.0, '#F9EBEA'), (1.0, '#641E16')],
                }),
                plgo.Surface({
                    'name': 'water',
                    'x': x,
                    'y': y,
                    'z': layer,
                    'colorscale': [(0.0, '#154360'), (1.0, '#85C1E9')],
                    'showscale': False,
                })
            ],
            'layout': plgo.Layout({
                'title': 'Simulation',
                'width': 800,
                'height': 800,
                'scene': plgo.Scene({
                    'zaxis': plgo.ZAxis({
                        'range': [0, 5]
                    })
                })
            }),
        })
    )

In [ ]:
# def upper_plot(land, water, Timestep):
#     mask = ~water[Timestep].astype(bool)
# #     mask = water[Timestep] < 0.0001
#     layer = land + water[Timestep]
#     layer[mask] = None
#     land_cm = col.LinearSegmentedColormap.from_list(
#         'land',
#         [(110/255, 44/255, 0), (246/255, 221/255, 204/255)][::-1]
#     )
#     water_cm = col.LinearSegmentedColormap.from_list(
#         'water',
#         [(27/255, 79/255, 144/255), (214/255, 234/255, 248/255)]
#     )
#     fig = plt.figure(figsize=(14,8))
#     ax = fig.add_subplot(111)
#     ax.imshow(land, cmap=land_cm, vmin=np.min(land), vmax=np.max(land))
#     ax.imshow(layer, cmap=water_cm, vmin=np.min(land), vmax=np.max(land))
#     plt.show()
    
def upper_plot(x, y, land, water, Timestep, save=False):
#     mask = ~water[Timestep].astype(bool)
    mask = water[Timestep] < 0.00001
    layer = land + water[Timestep]
    layer[mask] = None
    land_cm = col.LinearSegmentedColormap.from_list(
        'land',
        [(110/255, 44/255, 0), (246/255, 221/255, 204/255)][::-1]
    )
    water_cm = col.LinearSegmentedColormap.from_list(
        'water',
        [(27/255, 79/255, 144/255), (214/255, 234/255, 248/255)]
    )
    fig = plt.figure(figsize=(16,8))
    ax1 = fig.add_subplot(122)
    ax1.imshow(land, cmap=land_cm, vmin=np.min(land), vmax=np.max(land))
    ax1.imshow(layer, cmap=water_cm)#, vmin=0, vmax=0.05)
    ax2 = fig.add_subplot(121, projection='3d')
    ax2.plot_surface(x, y, land, cmap=land_cm)
    ax2.plot_surface(x, y, layer, cmap=water_cm)
    ax2.set_zlim(0, 5)
    if save:
        plt.savefig("images/test-{}.png".format(save))
    plt.show()

In [ ]:
def diff_plot(x, y, water1, water2, Timestep):
    water_cm = col.LinearSegmentedColormap.from_list(
        'water',
        [(27/255, 79/255, 144/255), (214/255, 234/255, 248/255)]
    )
    fig = plt.figure(figsize=(16,8))
    ax1 = fig.add_subplot(122)
    ax1.imshow(water1[Timestep] - water2[Timestep], cmap=water_cm)
    ax2 = fig.add_subplot(121, projection='3d')
    ax2.plot_surface(x, y, water1[Timestep] - water2[Timestep], cmap=water_cm)
    plt.show()

In [ ]:
def simple_automata_step(terra,water,alpha):
    assert(terra.shape==water.shape)
    levels = terra+water
    # Get the level transfer:
    transfer_x = levels[:,:-1]-levels[:,1:]
    transfer_x = np.minimum(water[:,:-1],transfer_x)
    transfer_x = np.maximum(-water[:,1:],transfer_x)
    #
    
    transfer_y = levels[:-1,:]-levels[1:,:]
    transfer_y = np.minimum(water[:-1,:],transfer_y)
    transfer_y = np.maximum(-water[1:,:],transfer_y)
    
    # Update water levels:
    nwater = water.copy()
    nwater[:,:-1] -= transfer_x*alpha
    nwater[:,1:]  += transfer_x*alpha
    nwater[:-1,:] -= transfer_y*alpha
    nwater[1:,:]  += transfer_y*alpha
    return nwater

In [ ]:
def inertial_automata_step(x_vec, y_vec, land, water, alpha, beta, gamma):
    assert(land.shape == water.shape)
    levels = land + water
    # Get the level transfer:
    transfer_x = levels[:,:-1] - levels[:,1:]
    transfer_x = np.minimum(water[:,:-1], transfer_x)
    transfer_x = np.maximum(-water[:,1:], transfer_x)
    #
    transfer_y = levels[:-1,:] - levels[1:,:]
    transfer_y = np.minimum(water[:-1,:], transfer_y)
    transfer_y = np.maximum(-water[1:,:], transfer_y)
    # Update intertial vectors
    nx_vec = beta * x_vec + gamma * transfer_x
    ny_vec = beta * y_vec + gamma * transfer_y
    #
    delta_x = transfer_x * alpha + nx_vec
    delta_x[delta_x < 0] = - np.minimum(water[:,1:][delta_x < 0]/4, -delta_x[delta_x < 0])
    delta_x[delta_x > 0] = np.minimum(water[:,:-1][delta_x > 0]/4, delta_x[delta_x > 0])
    #
    delta_y = transfer_y * alpha + ny_vec
    delta_y[delta_y < 0] = - np.minimum(water[1:,:][delta_y < 0]/4, -delta_y[delta_y < 0])
    delta_y[delta_y > 0] = np.minimum(water[:-1,:][delta_y > 0]/4, delta_y[delta_y > 0])
    # Update water levels:
    nwater = water.copy()
    nwater[:,:-1] -= transfer_x * alpha + nx_vec
    nwater[:,1:]  += transfer_x * alpha + nx_vec
    nwater[:-1,:] -= transfer_y * alpha + ny_vec
    nwater[1:,:]  += transfer_y * alpha + ny_vec

    return nx_vec, ny_vec, nwater

In [ ]:
def inertial_automata_step2(x_vec, y_vec, land, water, alpha, beta, gamma):
    assert(land.shape == water.shape)
    levels = land + water
    # Get the level transfer:
    transfer_x = levels[:,:-1] - levels[:,1:]
    transfer_x = np.minimum(water[:,:-1], transfer_x)
    transfer_x = np.maximum(-water[:,1:], transfer_x)
    #
    transfer_y = levels[:-1,:] - levels[1:,:]
    transfer_y = np.minimum(water[:-1,:], transfer_y)
    transfer_y = np.maximum(-water[1:,:], transfer_y)
    # Update intertial vectors
    nx_vec = beta * x_vec + gamma * transfer_x
    ny_vec = beta * y_vec + gamma * transfer_y
    #
    delta_x = transfer_x * alpha + nx_vec
    delta_x[delta_x < 0] = - np.minimum(water[:,1:][delta_x < 0]/4, -delta_x[delta_x < 0])
    delta_x[delta_x > 0] = np.minimum(water[:,:-1][delta_x > 0]/4, delta_x[delta_x > 0])
    #
    delta_y = transfer_y * alpha + ny_vec
    delta_y[delta_y < 0] = - np.minimum(water[1:,:][delta_y < 0]/4, -delta_y[delta_y < 0])
    delta_y[delta_y > 0] = np.minimum(water[:-1,:][delta_y > 0]/4, delta_y[delta_y > 0])
    # Update water levels:
    nwater = water.copy()
    nwater[:,:-1] -= delta_x
    nwater[:,1:]  += delta_x
    nwater[:-1,:] -= delta_y
    nwater[1:,:]  += delta_y

    return nx_vec, ny_vec, nwater

In [ ]:
def inertial_automata_step2(x_vec, y_vec, land, water, alpha, beta, gamma, k=0.00001):
    levels = land+water
    levelsb = np.zeros((levels.shape[0]+2,levels.shape[1]+2))
    levelsb[1:-1,1:-1] = levels
    levelsb[0,:] = float('inf')
    levelsb[-1,:] = float('inf')
    levelsb[:,0] = float('inf')
    levelsb[:,-1] = float('inf')
    # Get water level height difference on each direction
    delta1 = levels-levelsb[2:,1:-1]
    delta1 = np.min([delta1,water],axis=0)
    delta2 = levels-levelsb[1:-1,:-2]
    delta2 = np.min([delta2,water],axis=0)
    delta3 = levels-levelsb[:-2,1:-1]
    delta3 = np.min([delta3,water],axis=0)
    delta4 = levels-levelsb[1:-1,2:]
    delta4 = np.min([delta4,water],axis=0)
    # Filter small and negative discharges
    delta1[delta1<k] = 0
    delta2[delta2<k] = 0
    delta3[delta3<k] = 0
    delta4[delta4<k] = 0
    # Update inertial vectors
    nx_vec = beta * x_vec + gamma * (delta1-delta3)
    ny_vec = beta * y_vec + gamma * (delta4-delta2)
    # NOTE: remove innertia at the borders or we will losse water!
    nx_vec[:,-1] = 0
    nx_vec[:,0] = 0
    nx_vec[-1,:] = 0
    nx_vec[0,:] = 0
    ny_vec[:,-1] = 0
    ny_vec[:,0] = 0
    ny_vec[-1,:] = 0
    ny_vec[0,:] = 0
    # Multiply all differences by alpha to get the discharges:
    delta1 *= alpha
    delta3 *= alpha
    delta4 *= alpha
    delta2 *= alpha
    # Add the inertia to the discharges
    delta1[nx_vec>0] += nx_vec[nx_vec>0]
    delta2[ny_vec<0] += -ny_vec[ny_vec<0]
    delta3[nx_vec<0] += -nx_vec[nx_vec<0]
    delta4[ny_vec>0] += ny_vec[ny_vec>0]
    # Compute the total discharged water for each cell
    deltaT = delta1+delta2+delta3+delta4
    # As a cell cannot discharge more than its water level:
    coefs = water[deltaT>water]/deltaT[deltaT>water]
    delta1[deltaT>water] *= coefs
    delta2[deltaT>water] *= coefs
    delta3[deltaT>water] *= coefs
    delta4[deltaT>water] *= coefs
    # Transfer flow
    nwater = water.copy()
    nwater -= delta1+delta2+delta3+delta4
    nwater[1:,:] += delta1[:-1,:]
    nwater[:,:-1] += delta2[:,1:]
    nwater[:-1,:] += delta3[1:,:]
    nwater[:,1:] += delta4[:,:-1]
    # Return new water level
    return nx_vec, ny_vec, nwater

In [ ]:
def inertial_automata_step_8(x_vec, y_vec, xy_vec, yx_vec, land, water, alpha, beta, gamma):
    assert(land.shape == water.shape)
    levels = land + water
    # Get the level transfer:
    transfer_x = levels[:,:-1] - levels[:,1:]
    transfer_x = np.minimum(water[:,:-1], transfer_x)
    transfer_x = np.maximum(-water[:,1:], transfer_x)
    #
    transfer_y = levels[:-1,:] - levels[1:,:]
    transfer_y = np.minimum(water[:-1,:], transfer_y)
    transfer_y = np.maximum(-water[1:,:], transfer_y)
    #
    transfer_xy = levels[:-1,:-1] - levels[1:,1:]
    transfer_xy = np.minimum(water[:-1,:-1], transfer_xy)
    transfer_xy = np.maximum(-water[1:,1:], transfer_xy)
    #
    transfer_yx = levels[:-1,1:] - levels[1:,:-1]
    transfer_yx = np.minimum(water[:-1,1:], transfer_yx)
    transfer_yx = np.maximum(-water[1:,:-1], transfer_yx)
    # Update intertial vectors
    nx_vec = beta * x_vec + gamma * transfer_x
    nxy_vec = beta * xy_vec + gamma * transfer_xy
    ny_vec = beta * y_vec + gamma * transfer_y
    nyx_vec = beta * yx_vec + gamma * transfer_yx
    # Update water levels:
    nwater = water.copy()
    nwater[:,:-1] -= transfer_x * alpha + nx_vec
    nwater[:,1:]  += transfer_x * alpha + nx_vec
    nwater[:-1,:] -= transfer_y * alpha + ny_vec
    nwater[1:,:]  += transfer_y * alpha + ny_vec
    nwater[:-1,:-1] -= transfer_xy * alpha + nxy_vec
    nwater[1:,1:]   += transfer_xy * alpha + nxy_vec
    nwater[:-1,1:]  -= transfer_yx * alpha + nyx_vec
    nwater[1:,:-1]  += transfer_yx * alpha + nyx_vec
    return nx_vec, ny_vec, nxy_vec, nyx_vec, nwater

In [ ]:
size = 1000
alpha = 0.2

w = np.zeros((size, *land.shape))
w[0,:,:] = simple_automata_step(land, water, alpha)
for i in log_progress(range(size-1), every=1, name='Timesteps'):
    w[i+1,:,:] = simple_automata_step(land, w[i,:,:], alpha)

In [ ]:
res = 100
base = np.linspace(0, 5, res)
x, y = np.meshgrid(base, base)
# final_func = np.vectorize(lambda x, y: noise.pnoise2(x, y, octaves=4))
# land = final_func(x, y)
land = np.zeros(x.shape)
x_dir = np.zeros((res, res-1))
x_dir2 = np.zeros((res-1, res-1))
y_dir = np.zeros((res-1, res))
y_dir2 = np.zeros((res-1, res-1))

# water = np.zeros(land.shape)
# water[25:76,25:76] = 5 - land[25:76,25:76]
water = 10 * np.e ** -((x-2.5)**2 + (y-2.5)**2)

In [ ]:
size = 1000
alpha = 0.1
beta = 0.99
gamma = 0.05

w = np.zeros((size, *land.shape))
x_vec = np.zeros(x_dir.shape)
y_vec = np.zeros(y_dir.shape)
x_vec, y_vec, w[0,:,:] = inertial_automata_step(x_dir, y_dir, land, water, alpha, beta, gamma)
for i in log_progress(range(size-1), every=1, name='Timesteps'):
    x_vec, y_vec, w[i+1,:,:] = inertial_automata_step(x_vec, y_vec, land, w[i,:,:], alpha, beta, gamma)

In [ ]:
# size = 1000
# alpha = 0.1
# beta = 0.99
# gamma = 0.05

# w = np.zeros((size, *land.shape))
# x_vec = np.zeros(x_dir.shape)
# y_vec = np.zeros(y_dir.shape)
# xy_vec = np.zeros(x_dir2.shape)
# yx_vec = np.zeros(y_dir2.shape)
# x_vec, y_vec, xy_vec, yx_vec, w[0,:,:] = inertial_automata_step_8(
#     x_vec, y_vec, xy_vec, yx_vec, land, water, alpha, beta, gamma
# )
# for i in log_progress(range(size-1), every=1, name='Timesteps'):
#     x_vec, y_vec, xy_vec, yx_vec, w[i+1,:,:] = inertial_automata_step_8(
#         x_vec, y_vec, xy_vec, yx_vec, land, w[i,:,:], alpha, beta, gamma
#     )

In [ ]:
wdg.interact(upper_plot, Timestep=(0, size-1, 10), water=wdg.fixed(w), land=wdg.fixed(land), x=wdg.fixed(x), y=wdg.fixed(y))

In [ ]:
for i in range(size):
    upper_plot(x, y, land, w, i, "{:0>4}".format(i+1))

In [ ]:
def check_conservation(water_array):
    ref = np.add.reduce(water_array[0], axis=(0,1))
    error = np.max(np.add.reduce(water_array, axis=(1,2)) - ref)
    return np.log10(error) - np.log10(ref)

In [ ]:
check_conservation(w)

In [ ]:
res = 100
base = np.linspace(0, 5, res)
x, y = np.meshgrid(base, base)
# final_func = np.vectorize(lambda x, y: noise.pnoise2(x, y, octaves=4))
# land = final_func(x, y)
land = np.zeros(x.shape)
x_dir = np.zeros(land.shape)
y_dir = np.zeros(land.shape)

# water = np.zeros(land.shape)
# water[25:76,25:76] = 5 - land[25:76,25:76]
water = 10 * np.e ** -((x-2.5)**2 + (y-2.5)**2)

In [ ]:
size = 1000
alpha = 0.1
beta = 0.99
gamma = 0.05

w2 = np.zeros((size, *land.shape))
x_vec = np.zeros(x_dir.shape)
y_vec = np.zeros(y_dir.shape)
x_vec, y_vec, w2[0,:,:] = inertial_automata_step2(x_dir, y_dir, land, water, alpha, beta, gamma, 0)
for i in log_progress(range(size-1), every=1, name='Timesteps'):
    x_vec, y_vec, w2[i+1,:,:] = inertial_automata_step2(x_vec, y_vec, land, w2[i,:,:], alpha, beta, gamma, 0)

In [ ]:
wdg.interact(diff_plot, Timestep=(0, size-1, 10), water1=wdg.fixed(w), water2=wdg.fixed(w2), x=wdg.fixed(x), y=wdg.fixed(y))

In [ ]:
plot(x, y, land, w[200])